In [1]:
from tkinter import *
from tkinter import filedialog
from tkinter import ttk

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import xlwings as xw
import time

In [2]:
# Define a function to scroll to the bottom of the page
def scroll_to_bottom():
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
excel = ''
def browse():
    global excel
    excel = filedialog.askopenfilename(title = "Datei auswählen")

def go_dodo():
    global no_congress, no_excel
    try:
        no_congress.grid_forget()
    except:
        pass
    
    try:
        no_excel.grid_forget()
    except:
        pass
    
    if 'http' in str(excel):
        # retrieve url
        url = url_entry.get()
        # identify congress
        congress = ''
        congress_list = ['caclp', 'eccmid', 'medica', 'medlab']
        for con in congress_list:
            if con in url:
                congress = con

        # CACLP
        if congress == 'caclp':
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.maximize_window()
            time.sleep(1)
            accept_xpath = '/html/body/div[9]/div/div/div[2]/a[2]'
            accept = driver.find_element(By.XPATH, accept_xpath)
            accept.click()
            # Seite alphabetisch ordnen
            alpha = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[1]/div[2]/div/p[1]/a')
            alpha.click()
            # Anzahl der Seiten finden
            seitenanzahl_xpath = '/html/body/div[3]/div/div/div[1]/div[3]/div/a[2]'
            seitenanzahl = driver.find_element(By.XPATH, seitenanzahl_xpath)
            seitenanzahl_text = seitenanzahl.text
            if 'of' in seitenanzahl_text:
                seitenanzahl_text = seitenanzahl_text.replace('of', '').strip()
            seiten_zahl = int(seitenanzahl_text)

            namen_liste = []
            booth_liste = []
            country_liste = []
            link_list = []

            input_seite_xpath = '/html/body/div[3]/div/div/div[1]/div[3]/div/input'
            input_seite = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, input_seite_xpath)))
            input_seite = driver.find_element(By.XPATH, input_seite_xpath)

            for i in range(27, seiten_zahl+1):
                # input der Seitenzahl identifizieren
                time.sleep(2)
                input_seite = driver.find_element(By.XPATH, input_seite_xpath)
                input_seite.clear()
                input_seite.send_keys(i)
                input_seite.send_keys(Keys.ENTER)
                austeller = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[1]/div[3]')
                austeller_elements = austeller.find_elements(By.XPATH, ".//ul/li/a")
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(5)

                # Firmennamen ablesen, wenn diese gegeben sind. Ansonsten nicht
                for j in range(len(austeller_elements)):
                    time.sleep(1)
                    try:
                        link_url = austeller_elements[j].get_attribute("href")
                        driver.execute_script("window.open('" + link_url + "', '_blank');")
                    except:
                        austeller_elements = austeller.find_elements(By.XPATH, ".//ul/li/a")
                        link_url = austeller_elements[j].get_attribute("href")
                        driver.execute_script("window.open('" + link_url + "', '_blank');")
                    time.sleep(1)
                    new_window = driver.window_handles[-1]
                    driver.switch_to.window(new_window)
                    try:
                        firmenname = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/div[1]/h1')
                        name = firmenname.text
                        namen_liste.append(name)
                        booth = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/ul/li[1]/span')
                        booth_liste.append(booth.text)
                        country = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/ul/li[3]/span')
                        country_liste.append(country.text)
                        link_list.append(driver.current_url)
                        print(name)
                    except Exception as e:
                        print(e)
                        pass
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    time.sleep(1)

            # falls Angaben nicht vollständig sind, befindet sich in der country_Liste die Kontaktinfo
            for i in range(len(country_liste)):
                if ('@' in country_liste[i]) | ('htt' in country_liste[i])| ('www' in country_liste[i].lower()):
                    country_liste[i] = ''

            # falls keine Booth-Info gegeben ist, dann soll Zelle leer sein
            for i in range(len(booth_liste)):
                if ('Manufacturer' in booth_liste[i]) | ('Service' in booth_liste[i]) | ('Distributor' in booth_liste[i]):
                    booth_liste[i] = ''

            # Excel ausfüllen
            wb = xw.Book(excel)
            sheet = wb.sheets[0]

            start_row = 1040

            A_col_range = sheet.range('A{}:A{}'.format(start_row, start_row+len(namen_liste)))
            A_col_range.value = [[value] for value in namen_liste]

            I_col_range = sheet.range('I{}:I{}'.format(start_row, start_row+len(booth_liste)))
            I_col_range.value = [[value] for value in booth_liste]

            B_col_range = sheet.range('B{}:B{}'.format(start_row, start_row+len(country_liste)))
            B_col_range.value = [[value] for value in country_liste]

            # add hyperlinks
            for i in range(len(link_list)):
                row = start_row+i    
                cell_row, cell_column = row, 1
                cell_address = (cell_row, cell_column)
                sheet.range(cell_address).add_hyperlink(link_list[i], text_to_display=namen_liste[i])
                sheet.range(cell_address).api.Font.Bold = True
                sheet.range(cell_address).font.color = (0, 0, 0)
                sheet.range(cell_address).api.Font.Size = 14
                sheet.range(cell_address).api.Font.Name = 'Verdana'

        elif congress == 'eccmid':
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.maximize_window()
            time.sleep(1)
            accept_xpath = '/html/body/div[1]/div[2]/span[1]'
            accept = driver.find_element(By.XPATH, accept_xpath)
            accept.click()
            time.sleep(1)
            close_xpath = '//*[@id="close"]/img'
            close = driver.find_element(By.XPATH, close_xpath)
            close.click()

            # Get the initial page height
            initial_height = driver.execute_script("return document.body.scrollHeight")
            # Perform initial scroll
            scroll_to_bottom()
            # Wait for some time to let the content load
            time.sleep(2)
            # Keep scrolling until the page height stops increasing
            while True:
                # Scroll to the bottom of the page
                scroll_to_bottom()
                # Wait for some time to let the content load
                time.sleep(2)
                # Get the updated page height
                new_height = driver.execute_script("return document.body.scrollHeight")
                # Check if the page height has stopped increasing
                if new_height == initial_height:
                    break
                # Update the initial height for the next iteration
                initial_height = new_height

            div_xpath = '//*[@id="c41055"]/div/div[2]'

            # Find the div element
            div_element = driver.find_element(By.XPATH, div_xpath)

            # Find all the div elements within the div
            text_elements = div_element.find_elements(By.XPATH, ".//div")

            # Find all the a elements within the div (links)
            a_elements = div_element.find_elements(By.XPATH, ".//a")

            # Extract text and links
            text_list = [a.text for a in text_elements]
            link_list = [a.get_attribute("href") for a in a_elements]
            text_list = [text for text in text_list if text != '']

            # extract austeller und booth
            name = []
            booth = []
            for i in range(len(text_list)):
                name.append(text_list[i].split('\n')[0])
                booth.append(text_list[i].split('\n')[-1].split('Stand')[-1].strip())

            # Excel ausfüllen
            wb = xw.Book(excel)
            sheet = wb.sheets[0]

            A_col_range = sheet.range('A3:A{}'.format(3+len(name)))
            A_col_range.value = [[value] for value in name]

            I_col_range = sheet.range('I3:I{}'.format(3+len(booth)))
            I_col_range.value = [[value] for value in booth]

            # add hyperlinks
            for i in range(len(link_list)):
                row = 3+i    
                cell_row, cell_column = row, 1
                cell_address = (cell_row, cell_column)
                sheet.range(cell_address).add_hyperlink(link_list[i], text_to_display=name[i])
                sheet.range(cell_address).api.Font.Bold = True
                sheet.range(cell_address).font.color = (0, 0, 0)
                sheet.range(cell_address).api.Font.Size = 14
                sheet.range(cell_address).api.Font.Name = 'Verdana'


        elif congress == 'medica':
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.maximize_window()

            # Shadow Root, falls Programm es nicht findet, selber klicken
            shadow_root = driver.find_element(By.CSS_SELECTOR, "#usercentrics-root").shadow_root
            sel = '#uc-center-container > div.sc-kdBSHD.gsTflx > div > div.sc-dtBdUo.fzvttm > div > button.sc-dcJsrY.eyWWqo'
            accept = WebDriverWait(shadow_root, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,sel))) 
            accept.click()
            # Find the parent div element using its ID
            parent_div_id = 'vis-search-scroll-area'
            parent_div = driver.find_element(By.ID, parent_div_id)

            unique_elements = []

            # Maximum number of iterations to scroll down
            max_scroll_iterations = 100

            # Scroll down and collect div elements until the maximum number of iterations or no new elements are found
            for _ in range(max_scroll_iterations):
                prev_elements_count = len(unique_elements)

                # Scroll down the page to load more content
                scroll_down()
                time.sleep(1)

                # Wait for some time to let the new content load
                WebDriverWait(driver, 30).until(lambda d: len(d.find_elements(By.XPATH, './/div')) > prev_elements_count)

                child_div_class_name = 'searchresult-item'
                child_div_elements = parent_div.find_elements(By.CLASS_NAME, child_div_class_name)

                new_elements_found = False

                for div_element in child_div_elements:
                    element_text = div_element.text
                    # Check if the element's text is not already in the set before adding it to the list
                    if element_text not in unique_elements:
                        # Add the element's text to the set
                        unique_elements.append(element_text)
                        new_elements_found = True

                # If no new elements are found in this iteration, break the loop
                if not new_elements_found:
                    break

            unique_elements_set = list(unique_elements)

            name = [None]*len(unique_elements_set)
            name_search = [None]*len(unique_elements_set)
            country = [None]*len(unique_elements_set)
            hall = [None]*len(unique_elements_set)

            for i in range(len(unique_elements_set)):
                if 'Aussteller\n' in unique_elements_set[i]:
                    index_aussteller = unique_elements_set[i].index('Aussteller\n') + len('Aussteller\n')
                    unique_elements_set[i] = unique_elements_set[i][index_aussteller:]
                    while True:
                        if 'Aussteller\n' in unique_elements_set[i]:
                            index_aussteller = unique_elements_set[i].index('Aussteller\n') + len('Aussteller\n')
                            unique_elements_set[i] = unique_elements_set[i][index_aussteller:]
                        else:
                            break        
                    name[i] = unique_elements_set[i].split('\n')[0].strip()
                    name_search[i] = unique_elements_set[i].split('\n')[0].strip()
                    hall[i] = unique_elements_set[i].split('\n')[-1].strip()
                    land = unique_elements_set[i].split('\n')[1].split(',')[-1].strip()
                    if land == 'Republik':
                        land = unique_elements_set[i].split('\n')[1].split(',')[-2].strip()
                    country[i] = land
                else:
                    name[i] = unique_elements_set[i].split('\n')[-2].split(':')[-1].strip()
                    name_search[i] = unique_elements_set[i].split('\n')[0].strip()
                    hall[i] = unique_elements_set[i].split('\n')[-1].strip()
                    country[i] = ''
                # Überschreiben von name_search, falls es sich um Produkt handelt
                if 'Produkt\n' in unique_elements_set[i]:
                    name_search[i] = unique_elements_set[i].split('Produkt\n')[1].split('\n')[0].strip()

            # country name ger to eng
            country_dict = {'Frankreich': 'France',
                           'Korea': 'South Korea',
                           'Italien': 'Italy',
                           'Indien': 'India',
                           'Deutschland': 'Germany',
                           'Polen': 'Poland',
                           'Belgien': 'Belgium',
                           'Kanada': 'Canada',
                           'Schweiz': 'Switzerland',
                           'Österreich': 'Austria',
                           'Spanien': 'Spain',
                           'Türkei': 'Turkey',
                           'Großbritannien und Nordirland': 'UK',
                           'Luxemburg': 'Luxembourg',
                           'Islamische Republik': 'Islamic Republic',
                           'Dänemark': 'Denmark',
                           'Niederlande': 'Netherlands',
                           'Schweden': 'Sweden',
                           'Ägypten': 'Egypt',
                           'Tunesien': 'Tunesia',
                           'Finnland': 'Finland',
                           'Litauen': 'Lithuania',
                           'Estland': 'Estonia',
                            'Kroatien': 'Croatia',
                            'Ungarn': 'Hungary',
                            'Griechenland': 'Greece',
                            'Tschechien': 'Czech',
                            'Australien': 'Australia',
                            'Brasilien': 'Brazil',
                            'Slowenien': 'Slovenia',
                            'Norwegen': 'Norway'
                           }

            for i in range(len(country)):
                if country[i] in country_dict:
                    country[i] = country_dict[country[i]]    

            # Dopplungen entfernen
            drop_index = []
            for i in range(len(name)-1):
                if (name[i] == name[i+1]) & (country[i] == country[i+1]) & (hall[i] == hall[i+1]):
                    drop_index.append(i)

            name = [name[i] for i in range(len(name)) if i not in drop_index]
            name_search = [name_search[i] for i in range(len(name_search)) if i not in drop_index]
            country = [country[i] for i in range(len(country)) if i not in drop_index]
            hall = [hall[i] for i in range(len(hall)) if i not in drop_index]

            # Excel ausfüllen
            wb = xw.Book(excel)
            sheet = wb.sheets[0]

            A_col_range = sheet.range('A3:A{}'.format(3+len(name)))
            A_col_range.value = [[value] for value in name]

            B_col_range = sheet.range('B3:B{}'.format(3+len(country)))
            B_col_range.value = [[value] for value in country]

            url = []
            url_booth = []

            for n in name_search:
                # get xpath based on name
                target_text = n

                if "'" in target_text:
                    target_text = target_text.split("'")[0]

                # Find the element using its text with an XPath query
                xpath_query = f"//*[contains(text(), '{target_text}')]"
                element = driver.find_element(By.XPATH, xpath_query)

                # Evaluate the XPath of the element using JavaScript execution
                element_xpath = driver.execute_script(
                    "function getElementXPath(element) {"
                    "    if (element && element.id) {"
                    "        return '//' + element.tagName.toLowerCase() + '[@id=\"' + element.id + '\"]';"
                    "    } else {"
                    "        return getElementXPath(element.parentElement) + '/' + element.tagName.toLowerCase();"
                    "    }"
                    "}"
                    "return getElementXPath(arguments[0]);",
                    element
                )
                # Open the link in a new tab using CTRL+click
                action = ActionChains(driver)
                action.key_down(Keys.CONTROL).click(element).key_up(Keys.CONTROL).perform()

                # Switch to the newly opened tab
                driver.switch_to.window(driver.window_handles[1])

                # Append the current URL of the newly opened tab to the url list
                url.append(driver.current_url)

                # Scroll up by sending the "PAGE_UP" key using Keys class
                body = driver.find_element(By.TAG_NAME, "body")
                body.send_keys(Keys.PAGE_UP)
                time.sleep(1)

                try:
                    booth = driver.find_element(By.XPATH, '//*[@id="finder-profile"]/div/div/section/div/div/div[1]/div[2]/ul/li/a/div/button/div/span[2]')

                    # Open the link in a new tab using CTRL+click
                    action = ActionChains(driver)
                    action.key_down(Keys.CONTROL).click(booth).key_up(Keys.CONTROL).perform()      

                    time.sleep(1)
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(1)
                    url_booth.append(driver.current_url)

                    # Close the newly opened tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[1])
                except:
                    url_booth.append('')

                # Close the opened tab
                driver.close()
                # Switch back to the original tab
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(1)

            # add hyperlinks
            for i in range(len(url)):
                row = 3+i
                cell_row, cell_column = row, 9
                cell_address = (cell_row, cell_column)
                sheet.range(cell_address).add_hyperlink(url_booth[i], text_to_display=hall[i])

                cell_row, cell_column = row, 1
                cell_address = (cell_row, cell_column)
                sheet.range(cell_address).add_hyperlink(url[i], text_to_display=name[i])
                sheet.range(cell_address).api.Font.Bold = True
                sheet.range(cell_address).font.color = (0, 0, 0)
                sheet.range(cell_address).api.Font.Size = 14
                sheet.range(cell_address).api.Font.Name = 'Verdana'


        elif congress == 'medlab':
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.maximize_window()

            accept_xpath = '//*[@id="modal-container-1705325769754-0.07986755175320881"]/div/div/div/div[1]/button[2]/span/span/span'
            accept = driver.find_element(By.XPATH, accept_xpath)
            accept.click()

            # Get the initial page height
            initial_height = driver.execute_script("return document.body.scrollHeight")
            # Perform initial scroll
            scroll_to_bottom()
            # Wait for some time to let the content load
            time.sleep(2)
            # Keep scrolling until the page height stops increasing
            while True:
                # Scroll to the bottom of the page
                scroll_to_bottom()
                # Wait for some time to let the content load
                time.sleep(2)
                # Get the updated page height
                new_height = driver.execute_script("return document.body.scrollHeight")
                # Check if the page height has stopped increasing
                if new_height == initial_height:
                    break
                # Update the initial height for the next iteration
                initial_height = new_height

            div_xpath = '//*[@id="__next"]/div[3]/div/div[3]/div[2]/div/div/div/div[2]'

            # Find the div element
            div_element = driver.find_element(By.XPATH, div_xpath)

            # Find all the a elements within the div
            a_elements = div_element.find_elements(By.XPATH, ".//a")

            # Extract text and links
            text_list = [a.text for a in a_elements]
            link_list = [a.get_attribute("href") for a in a_elements]

            # extract austeller und booth
            name = []
            booth = []
            for i in range(len(text_list)):
                name.append(text_list[i].split('\n')[0])
                booth.append(text_list[i].split('\n')[-1])

            # Excel ausfüllen
            wb = xw.Book(excel)
            sheet = wb.sheets[0]

            A_col_range = sheet.range('A3:A{}'.format(3+len(name)))
            A_col_range.value = [[value] for value in name]

            I_col_range = sheet.range('I3:I{}'.format(3+len(booth)))
            I_col_range.value = [[value] for value in booth]

            # add hyperlinks
            for i in range(len(link_list)):
                row = 3+i    
                cell_row, cell_column = row, 1
                cell_address = (cell_row, cell_column)
                sheet.range(cell_address).add_hyperlink(link_list[i], text_to_display=name[i])
                sheet.range(cell_address).api.Font.Bold = True
                sheet.range(cell_address).font.color = (0, 0, 0)
                sheet.range(cell_address).api.Font.Size = 14
                sheet.range(cell_address).api.Font.Name = 'Verdana'

            # Iterate over the links
            austeller_text = []
            for link in link_list:
                # Open a new tab and switch to it
                driver.execute_script("window.open('', '_blank');")
                driver.switch_to.window(driver.window_handles[1])
                # Navigate to the link
                driver.get(link)
                # Wait for some time to let the content load (you can adjust this time)
                time.sleep(2)
                try:
                    # Extract text from the webpage
                    webpage_text = driver.find_element_by_tag_name('body').text
                    austeller_text.append(webpage_text)
                except NoSuchWindowException:
                    # Handle the case where the window is already closed
                    print(f"Window for {link} is already closed.")
                # Close the tab
                driver.close()
                # Switch back to the original tab
                driver.switch_to.window(driver.window_handles[0])

            land = []
            for i in range(len(austeller_text)):
                land.append(austeller_text[i].split('Country\n')[-1].split('\n')[0])

            B_col_range = sheet.range('B3:B{}'.format(3+len(land)))
            B_col_range.value = [[value] for value in land]


        else:
            no_congress = Label(window, text = '\nUnbekannter Link, melde dich gerne bei LQu :)', bg = '#eeeee4', font=('Ink free',12))
            no_congress.grid(row=8, column=1, columnspan=3)
        
    # falls excel == '', Meldung im GUI, dass Tabelle ausgewählt sein muss
    else:
        no_excel = Label(window, text = '\nBitte wähle noch die Vorlage aus.', bg = '#eeeee4', font=('Ink free',12))
        no_excel.grid(row=8, column=1, columnspan=3)



# User Interface
window = Tk()
window.title('Hallo DM')
window.geometry('800x400')
window.iconbitmap('O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo_icon.ico')
window.config(bg='#eeeee4')

# Text Erklärung
wozu = Label(window, text =
             'Hallo liebes Marketing!\n Mit diesem Programm werden die Ausstellerlisten erstellt.',
            bg = '#eeeee4', font = ('Ink free',15))
wozu.grid(row=1, column=1, columnspan=3)
excel = Label(window, text = 'Wähle zuerst die Vorlage für die Ausstellerliste aus\n', bg = '#eeeee4', font=('Ink free',12))
excel.grid(row=2, column=1, columnspan=3)
browse = Button(window, text = 'Browse', bg = '#869287', font=('Ink free',10,'bold'), command = browse)
browse.grid(row=3, column=2)
anweisung = Label(window, text = '\nUnd dann gib den Link ein:\n', bg = '#eeeee4', font = ('Ink free',12))
anweisung.grid(row=4, column=1, columnspan=3)
url_entry = Entry(window, font=('Arial', 10), width = 60)
url_entry.grid(row=5, column=1, columnspan=3)
leer = Label(window, text='', bg = '#eeeee4')
leer.grid(row=6, column=1)
go = Button(window, text = '  Go  ', bg = '#869287', font = ('Ink free',12,'bold'), command = go_dodo)
go.grid(row=7, column=1, columnspan=3)


# Logo
from PIL import ImageTk, Image
frame = Frame(window, width=10, height=10)
frame.grid(row=1, column=0)
img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((100,100)), 
                         master = window)
label = Label(frame, image = img, bg='#eeeee4')
label.pack()

window.mainloop()